In [ ]:
import pymesh
import numpy as np
import pandas as pd
import math
import glob

def distance3D(a, b):
    return math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)

fs_landmarks = sorted(glob.glob("../../Datos/data_artificial_cites/landmarks_reales/*.pts"))
fs_mesh = sorted(glob.glob("./out/torsos/*.ply"))

if np.shape(fs_landmarks) != np.array(fs_mesh):
    exit(0)

for landmarks_f, mesh_f in zip(fs_landmarks[:], fs_mesh[:]):
    print mesh_f+"  "+landmarks_f
    landmarks = pd.read_csv(landmarks_f,header=None,index_col=[0],skiprows=range(2),delimiter=r"\s+")
    mesh = pymesh.load_mesh(mesh_f)
    ls0 = landmarks.iloc[0].tolist()
    ls1 = landmarks.iloc[4].tolist()
    ls2 = landmarks.iloc[9].tolist()
    ls3 = landmarks.iloc[10].tolist()
    head = landmarks.iloc[14].tolist()
    foot_left = landmarks.iloc[7].tolist()
    foot_right = landmarks.iloc[6].tolist()
    knee_left = landmarks.iloc[8].tolist()
    knee_right = landmarks.iloc[5].tolist()
    center_knee = np.array([knee_left,knee_right]).mean(0)

    semi_landmark = np.array([ls1,ls2]).mean(0)
    distancias = []
    for v in mesh.vertices:
        distancias.append(distance3D(semi_landmark,v))
    distancias = np.array(distancias)
    centro = mesh.vertices[distancias.argmin()]
    distancias.argmin()

    delta = [0.4,0.0003,0.4]
    xdot = (head[0]*center_knee[0] + head[1]*center_knee[1] + head[2]*center_knee[2])
    norma = math.sqrt(head[0]**2+head[1]**2+head[2]**2) * math.sqrt(center_knee[0]**2+center_knee[1]**2+center_knee[2]**2)
    angle_head = (math.cos(xdot/norma) ** -1)
    angle_head_r = math.radians(90 - math.degrees(angle_head))
    q = pymesh.misc.Quaternion(quat=[1,-angle_head_r, 0,0])
    p1 = centro + delta
    p2 =centro  -  delta
    
    d = pymesh.meshutils.generate_icosphere(0.01,centro)
    pymesh.save_mesh("out/landk.ply", d)
    d

    c = pymesh.meshutils.generate_box_mesh(p2,p1 ,1)

    matrix_translate = [[1,0,0,0],[0,1,0,0.02],[0,0,1,0],[0,0,0,1]]

    alpha = math.pi /8
    #matrix = [[1,0,0],[0,math.cos(alpha),-math.sin(alpha)],[0,math.sin(alpha),math.cos(alpha)],[0,0,0]]
    matrix = [[1,0,0,0],[0,math.cos(alpha),-math.sin(alpha),0],[0,math.sin(alpha),math.cos(alpha),0],[0,0,0,1]]

    for i,v in enumerate(c.vertices_ref):
        c.vertices_ref[i] = np.dot(matrix,[v[0],v[1],v[2],1])[:-1] 
        #c.vertices_ref[i] = np.dot(matrix,v)[:-1] 
    pymesh.save_mesh("out/fetas/{}-feta-{}.ply".format(0,mesh_f.split("/")[-1]),c)
    #new_mesh = c

    for j in range(1,22):
        for i,v in enumerate(c.vertices_ref):
            c.vertices_ref[i] = np.dot(matrix_translate,[v[0],v[1],v[2],1])[:-1] 
            #c.vertices_ref[i] = np.dot(matrix,v)[:-1] 
            #new_mesh = pymesh.boolean(new_mesh, c , operation="union",engine="auto")
        pymesh.save_mesh("out/fetas/{}-feta-{}.ply".format(j,mesh_f.split("/")[-1]), c)

    new_mesh = pymesh.load_mesh("out/fetas/{}-feta-{}.ply".format(0,mesh_f.split("/")[-1]))
    for i in range(1,22):
        temp= pymesh.load_mesh("out/fetas/{}-feta-{}.ply".format(i,mesh_f.split("/")[-1]))
        new_mesh = pymesh.boolean(new_mesh, temp , operation="union",engine="auto")

    output_mesh = pymesh.boolean(new_mesh, mesh, operation="intersection",engine="auto")
    pymesh.save_mesh("out/torsos_feteados/feteado-{}".format(mesh_f.split("/")[-1]), output_mesh)

./out/torsos/model_mesh_001.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_001.pts
./out/torsos/model_mesh_002.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_002.pts
./out/torsos/model_mesh_004.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_004.pts
./out/torsos/model_mesh_005.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_005.pts
./out/torsos/model_mesh_006.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_006.pts
./out/torsos/model_mesh_009.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_009.pts
./out/torsos/model_mesh_010.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_010.pts
./out/torsos/model_mesh_011.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_011.pts
./out/torsos/model_mesh_012.ply  ../../Datos/data_artificial_cites/landmarks_reales/model_mesh_012.pts
./out/torsos/model_mesh_013.ply  ../../Datos/data_artificial_cites/landma